In [73]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances



# Load dataset
columns = ['movieID','userID','Ratings']
users_movies = pd.read_csv('exampledataset/data.txt', delimiter=',', names=columns)
users_movies.head()


# generate matrix

rating_matrix = users_movies.pivot(index='userID', columns='movieID',values='Ratings')

rating_matrix.fillna(0, inplace=True)

rating_matrix.head(35)
# item-based recommendation system

M = np.asarray([[3,7,4,9,9,7], 
                [7,0,5,3,8,8],
               [7,5,5,0,8,4],
               [5,6,8,5,9,8],
               [5,8,8,8,10,9],
               [7,7,0,4,7,8]])
M=pd.DataFrame(M)

In [77]:
def findSimilarItemsForAnItem(item_id, dataset,n_neighbors):
    similarities = []
    indices = []
    dataset = dataset.T
    
    knn_model = NearestNeighbors(metric='cosine',algorithm='brute')
    knn_model.fit(dataset)
    distances, indices = knn_model.kneighbors(dataset.iloc[item_id-1,:].values.reshape(1,-1), n_neighbors=n_neighbors+1)
    similarities = 1-distances.flatten();
    return similarities, indices


In [78]:
def predictItemsToUser(user_id, item_id, dataset, n_neighbors):
    prediction = 0
    weighted_sum = 0
    similarities, indices = findSimilarItemsForAnItem(item_id,dataset,4)
    sum_weight = np.sum(similarities)-1
    
    product = 1
    
    for i in range(0, len(indices.flatten())):
        if (i!=item_id+1):
            
            product = dataset.iloc[user_id, indices.flatten()[i]]*similarities[i]
            weighted_sum = weighted_sum + product
        prediction = weighted_sum/sum_weight
    
        #prediction = 0 if prediction <=0 else prediction
        #prediction = 5 if prediction > 5 else prediction
        print('\nPredicting Rating of User {0} for item {1}: {2}'.format(user_id, item_id, prediction))
        return prediction

prediction = predictItemsToUser(1,3,M,4)


Predicting Rating of User 1 for item 3: 1.470424566185883
